In [ ]:
import pandas as pd
from io import StringIO
import sys
import os
import csv
from sqlalchemy import create_engine,text
import numpy as np

# Add parent directory to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.db_connection import DatabaseConnection


db_connection = DatabaseConnection()

def read_row_data_from_bronze() -> pd.DataFrame :


        query = """
                SELECT * FROM bronze.covid where ingested_at = (select max(ingested_at) from bronze.covid);
        """
        return db_connection.read_dataframe_from_db(query)




row_data = read_row_data_from_bronze()

In [73]:
row_data.shape

(4011, 15)

In [74]:

def drop_columns(df: pd.DataFrame , columns_list: list) -> pd.DataFrame:
    df.drop(columns=columns_list , inplace=True)
    return df
    


columns=["fips" , "admin2" ,"combined_key" , "last_update",'lat','long_'] 


row_data = drop_columns(row_data , columns)




In [75]:
row_data.head(10)


,province_state,country_region,confirmed,deaths,recovered,active,incident_rate,case_fatality_ratio,ingested_at
0,None,Afghanistan,52586,2211,41727,8648,135.084102,4.204541,2021-01-02
1,None,Albania,58991,1190,34353,23448,2049.864480,2.017257,2021-01-02
2,None,Algeria,100159,2769,67611,29779,228.407338,2.764604,2021-01-02
3,None,Andorra,8166,84,7463,619,10568.821588,1.028655,2021-01-02
4,None,Angola,17608,407,11189,6012,53.574686,2.311449,2021-01-02
5,None,Antigua and Barbuda,159,5,148,6,162.364186,3.144654,2021-01-02
6,None,Argentina,1634834,43375,1447092,144367,3617.227335,2.653175,2021-01-02
7,None,Armenia,159798,2836,143640,13322,5392.689204,1.774741,2021-01-02
8,Australian Capital Territory,Australia,118,3,114,1,27.563653,2.542373,2021-01-02
9,New South Wales,Australia,4958,54,0,4904,61.074156,1.089149,2021-01-02


In [76]:
row_data.shape

(4011, 9)

In [77]:

row_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4011 entries, 0 to 4010
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   province_state       3833 non-null   object 
 1   country_region       4011 non-null   object 
 2   confirmed            4011 non-null   int64  
 3   deaths               4011 non-null   int64  
 4   recovered            4011 non-null   int64  
 5   active               4011 non-null   int64  
 6   incident_rate        3922 non-null   float64
 7   case_fatality_ratio  3963 non-null   float64
 8   ingested_at          4011 non-null   object 
dtypes: float64(2), int64(4), object(3)
memory usage: 282.1+ KB


In [78]:


def drop_null_values(df: pd.DataFrame , columns_list: list) -> pd.DataFrame:
    df.dropna(subset=columns_list , inplace=True)
    return df
    


columns=("country_region" , "confirmed" , "deaths","recovered","active","incident_rate" , "case_fatality_ratio")

row_data = drop_null_values(row_data , columns)



In [79]:
row_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3915 entries, 0 to 4010
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   province_state       3743 non-null   object 
 1   country_region       3915 non-null   object 
 2   confirmed            3915 non-null   int64  
 3   deaths               3915 non-null   int64  
 4   recovered            3915 non-null   int64  
 5   active               3915 non-null   int64  
 6   incident_rate        3915 non-null   float64
 7   case_fatality_ratio  3915 non-null   float64
 8   ingested_at          3915 non-null   object 
dtypes: float64(2), int64(4), object(3)
memory usage: 305.9+ KB


In [80]:

def drop_negtive_values(df:pd.DataFrame , columns_list : list) -> pd.DataFrame:
    for column in columns_list:
        df = df[df[column] >= 0]
    
    return df
        
columns = ["deaths" , "confirmed" , "recovered" , "active" , "incident_rate"]

row_data = drop_negtive_values(row_data , columns)

In [81]:
row_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3915 entries, 0 to 4010
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   province_state       3743 non-null   object 
 1   country_region       3915 non-null   object 
 2   confirmed            3915 non-null   int64  
 3   deaths               3915 non-null   int64  
 4   recovered            3915 non-null   int64  
 5   active               3915 non-null   int64  
 6   incident_rate        3915 non-null   float64
 7   case_fatality_ratio  3915 non-null   float64
 8   ingested_at          3915 non-null   object 
dtypes: float64(2), int64(4), object(3)
memory usage: 305.9+ KB


In [82]:
print(row_data["case_fatality_ratio"].max())
print(row_data["case_fatality_ratio"].min())
print(row_data["incident_rate"].max())
print(row_data["incident_rate"].min())


29.03379343169919
0.0
27388.21976571523
0.0


In [83]:



def format_locations (df: pd.DataFrame , cols: list) -> pd.DataFrame:
    for country in cols:
        df[country] = (
            row_data[country].str.strip()
                                      .str.lower()
                                      .str.title()
        )

    return df


columns_locations = ["province_state" , "country_region"]
row_data = format_locations(row_data , columns_locations)








In [84]:
row_data.head(10)

,province_state,country_region,confirmed,deaths,recovered,active,incident_rate,case_fatality_ratio,ingested_at
0,None,Afghanistan,52586,2211,41727,8648,135.084102,4.204541,2021-01-02
1,None,Albania,58991,1190,34353,23448,2049.864480,2.017257,2021-01-02
2,None,Algeria,100159,2769,67611,29779,228.407338,2.764604,2021-01-02
3,None,Andorra,8166,84,7463,619,10568.821588,1.028655,2021-01-02
4,None,Angola,17608,407,11189,6012,53.574686,2.311449,2021-01-02
5,None,Antigua And Barbuda,159,5,148,6,162.364186,3.144654,2021-01-02
6,None,Argentina,1634834,43375,1447092,144367,3617.227335,2.653175,2021-01-02
7,None,Armenia,159798,2836,143640,13322,5392.689204,1.774741,2021-01-02
8,Australian Capital Territory,Australia,118,3,114,1,27.563653,2.542373,2021-01-02
9,New South Wales,Australia,4958,54,0,4904,61.074156,1.089149,2021-01-02


In [85]:
def standlized_null_values(row_data:pd.DataFrame , colums_list:list)->pd.DataFrame:
    for column in columns_list:
        row_data[column] = row_data[column].replace(to_replace=['None','Unknown','','Niue'],value='None')

    return row_data



columns_list = ['province_state']
row_data = standlized_null_values(row_data , columns_list)



In [86]:
row_data['province_state']

0                   None
1                   None
2                   None
3                   None
4                   None
              ...       
4006                None
4007                None
4008                None
4009                None
4010    Pitcairn Islands
Name: province_state, Length: 3915, dtype: object

In [87]:
db_connection.load_dataframe_into_db(row_data,"silver","covid")

In [88]:
row_data

,province_state,country_region,confirmed,deaths,recovered,active,incident_rate,case_fatality_ratio,ingested_at
0,None,Afghanistan,52586,2211,41727,8648,135.084102,4.204541,2021-01-02
1,None,Albania,58991,1190,34353,23448,2049.864480,2.017257,2021-01-02
2,None,Algeria,100159,2769,67611,29779,228.407338,2.764604,2021-01-02
3,None,Andorra,8166,84,7463,619,10568.821588,1.028655,2021-01-02
4,None,Angola,17608,407,11189,6012,53.574686,2.311449,2021-01-02
...,...,...,...,...,...,...,...,...,...
4006,None,Ukraine,0,0,0,0,0.000000,0.000000,2021-01-02
4007,None,Nauru,0,0,0,0,0.000000,0.000000,2021-01-02
4008,None,New Zealand,0,0,0,0,0.000000,0.000000,2021-01-02
4009,None,Tuvalu,0,0,0,0,0.000000,0.000000,2021-01-02


{'loaded_data': 4011, 'rows_transformed': 3915, 'status': 'success'}